As a starting point, we have to read in all dataframes that have the word 'acquisition' and merge them into one dataframe.  Each of these CSV files contains info about a different bank.

In [1]:
import pandas as pd
import glob

acquisition_files = glob.glob("*acquisitions.csv")

acquisitions_df = [pd.read_csv(filename) for filename in acquisition_files]
acquisitions_df = pd.concat(acquisitions_df, axis=0, ignore_index=True)

Next, we'll read in the data for foreclosed loans and merge them with acquisitions.  The merge will have to be a left join, since not all acquisitions were foreclosed.

In [10]:
foreclosed_df = pd.read_csv("foreclosed_loans.csv")
all_loans = acquisitions_df.merge(foreclosed_df, how="left")

all_loans.head()

,loan_id,orig_channel,seller_name,orig_int_rate,original_upb,original_loan_term,orig_date,first_pymt_date,orig_ltv,orig_cltv,...,prop_state,zip_code,primary_insurance_pct,product_type,coborrower_credit_score,mortgage_insurance_type,relo_mortgage_indicator,current_delq_status,foreclosure_date,foreclosure_flag
0,100113431559.00,B,"FLAGSTAR BANK, FSB",6.38,73000.00,360.00,01/2005,03/2005,80.00,80.00,...,MI,492.00,nan,FRM,nan,nan,N,1.00,NaN,nan
1,100113444570.00,B,"FLAGSTAR BANK, FSB",5.75,70000.00,360.00,02/2005,04/2005,72.00,72.00,...,MO,656.00,nan,FRM,761.00,nan,N,1.00,NaN,nan
2,100167305688.00,C,"FLAGSTAR BANK, FSB",6.12,180000.00,240.00,01/2005,03/2005,58.00,58.00,...,CA,934.00,nan,FRM,nan,nan,N,1.00,NaN,nan
3,100213557668.00,B,"FLAGSTAR BANK, FSB",5.38,122000.00,180.00,12/2004,02/2005,85.00,85.00,...,MI,481.00,6.00,FRM,675.00,1.00,N,1.00,NaN,nan
4,100259899553.00,C,"FLAGSTAR BANK, FSB",5.38,206000.00,180.00,02/2005,04/2005,53.00,53.00,...,CA,900.00,nan,FRM,nan,nan,N,1.00,NaN,nan


The **foreclosure_flag** column has a value of '1' when a loan was foreclosed.  It will make things easier later if all NaN values (i.e., loans that were not foreclosed) had a value of '0' instead.

In [11]:
all_loans.foreclosure_flag.loc[all_loans.foreclosure_flag.isnull()] = 0

/usr/local/Cellar/jupyterlab/2.1.2/libexec/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
all_loans.foreclosure_flag.unique()

array([0., 1.])

Let's get a quick idea of the features of each column.

In [13]:
# suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

all_loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231497 entries, 0 to 231496
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   loan_id                  231497 non-null  float64
 1   orig_channel             231497 non-null  object 
 2   seller_name              231497 non-null  object 
 3   orig_int_rate            231497 non-null  float64
 4   original_upb             231497 non-null  float64
 5   original_loan_term       231497 non-null  float64
 6   orig_date                231497 non-null  object 
 7   first_pymt_date          231497 non-null  object 
 8   orig_ltv                 231497 non-null  float64
 9   orig_cltv                231481 non-null  float64
 10  num_borrowers            231496 non-null  float64
 11  orig_dti                 211948 non-null  float64
 12  credit_score             229423 non-null  float64
 13  first_time_buyer         231497 non-null  object 
 14  loan

Now, let's drop columns that we likely don't need to do the analysis.  We will remove variables that only apply to a subset of 
* loan_id serves no purpose
* It might be interesting to see whether there's a trend in when the loans were originated (approved and given), but first_pymnt_date might be redundant
* For this analysis, we won't look at zip_code
* current_delq_status contains no unique values
* primary_insurance_pct and mortgage_insurance_type only have values for ~10% of records

In [14]:
# drop unimportant variables
drop_cols = ['loan_id', 'first_pymt_date', 'zip_code',
             'current_delq_status', 'primary_insurance_pct',
            'mortgage_insurance_type']

all_loans = all_loans.drop(drop_cols, axis=1)
all_loans.head()

,orig_channel,seller_name,orig_int_rate,original_upb,original_loan_term,orig_date,orig_ltv,orig_cltv,num_borrowers,orig_dti,...,loan_purpose,property_type,number_units,occ_type,prop_state,product_type,coborrower_credit_score,relo_mortgage_indicator,foreclosure_date,foreclosure_flag
0,B,"FLAGSTAR BANK, FSB",6.38,73000.00,360.00,01/2005,80.00,80.00,1.00,41.00,...,C,SF,1.00,P,MI,FRM,nan,N,NaN,0.00
1,B,"FLAGSTAR BANK, FSB",5.75,70000.00,360.00,02/2005,72.00,72.00,2.00,34.00,...,C,SF,1.00,P,MO,FRM,761.00,N,NaN,0.00
2,C,"FLAGSTAR BANK, FSB",6.12,180000.00,240.00,01/2005,58.00,58.00,1.00,45.00,...,C,MH,1.00,P,CA,FRM,nan,N,NaN,0.00
3,B,"FLAGSTAR BANK, FSB",5.38,122000.00,180.00,12/2004,85.00,85.00,2.00,38.00,...,C,SF,1.00,P,MI,FRM,675.00,N,NaN,0.00
4,C,"FLAGSTAR BANK, FSB",5.38,206000.00,180.00,02/2005,53.00,53.00,1.00,50.00,...,C,SF,3.00,P,CA,FRM,nan,N,NaN,0.00


# What factors contribute to likelihood of foreclosure?

What we want to explore is which factors contribute the most to foreclosure.  We want to know what patterns exist that increase the likelihood that a loan will be foreclosed.  In other words, we want to hone in on the most important **predictor variables**.

As a starting point, let's look at 3 factors that banks usually look at to determine whether a borrower is credit worthy:

### 1. Credit score
From Investopedia:

> A credit score is a number ranging from 300-850 that depicts a consumer's creditworthiness. The higher the credit score, the more attractive the borrower.


### 2. Loan-to-value ratio (LTV)
This represents the total \\$ value of the mortgage divided by the \\$ appraised value of the house. For example, if you buy a house for $100,000 and put \\$20k down, you will need a mortgage of \\$80k. The LTV of this purchase would be 80\% (80k/100k).

If the LTV is higher, that means that you have less equity in the house. Higher LTVs are
traditionally considered to be riskier.


### 3. Debt-to-Income ratio (DTI)
DTI stands for Debt to Income. This represents total debt payments the borrower needs to
make in a month divided by their monthly gross income (including the mortgage payment
itself).

For Example – If you have a \\$500 Car Payment and a \\$2000 Mortgage and gross income of
\\$5000 a month, your DTI would be 50% (\\$2500 ÷ \\$5000). Typically, a higher DTI is riskier.

Let's take a look at some summary stats comparing the difference between these foreclose loans and non-foreclosed loans.

Then, let's create some plots to get a visual feel for the data.

In [7]:
# summarize all loans
all_loans.describe()

,orig_int_rate,original_upb,original_loan_term,orig_ltv,orig_cltv,num_borrowers,orig_dti,credit_score,number_units,coborrower_credit_score,foreclosure_flag
count,231497.00,231497.00,231497.00,231497.00,231481.00,231496.00,211948.00,229423.00,231497.00,103196.00,231497.00
mean,5.68,166677.57,313.79,68.31,70.03,1.58,36.23,721.34,1.04,729.19,0.04
std,0.36,83467.86,77.77,17.21,17.97,0.51,13.18,57.99,0.24,57.06,0.20
min,4.00,9000.00,84.00,1.00,1.00,1.00,1.00,384.00,1.00,391.00,0.00
25%,5.50,100000.00,240.00,58.00,60.00,1.00,26.00,679.00,1.00,688.00,0.00
50%,5.75,150000.00,360.00,73.00,74.00,2.00,36.00,727.00,1.00,738.00,0.00
75%,5.88,220000.00,360.00,80.00,80.00,2.00,46.00,771.00,1.00,777.00,0.00
max,9.00,692000.00,360.00,97.00,193.00,5.00,64.00,850.00,4.00,842.00,1.00


In [8]:
# summarize non-foreclosed loans
good_loans_stats = all_loans[all_loans.foreclosure_flag==0].describe()

# drop useless columns
good_loans_stats = good_loans_stats.drop(['loan_id',
                                          'zip_code',
                                          'foreclosure_flag'], 
                                         axis=1)

good_loans_stats

KeyError: "['loan_id' 'zip_code'] not found in axis"

In [ ]:
# summarize foreclosed loans only
bad_loans_stats = all_loans[all_loans.foreclosure_flag==1].describe()

# drop useless columns
bad_loans_stats = bad_loans_stats.drop(['loan_id',
                                          'zip_code',
                                          'foreclosure_flag'], 
                                         axis=1)

bad_loans_stats

Let's take a look at 

In [ ]:
# compare difference between dataframes
compare_stats = bad_loans_stats.div(good_loans_stats).sub(1).mul(100)
compare_stats

Now let's plot them.  Let's add the magic function *%matplotlib inline* to make sure that the images are stored in this notebook.

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

credit_plot = sns.lmplot("credit_score", 
                         "orig_dti",
                         hue="foreclosure_flag",
                         data=all_loans,
                         fit_reg=False)
plt.show()

In [ ]:
# histogram of credit score for non-foreclosed loans
all_loans[all_loans.foreclosure_flag == 0].loc[:, 'credit_score'].hist()

In [ ]:
# histogram of credit score for foreclosed loans
all_loans[all_loans.foreclosure_flag == 1].loc[:, 'credit_score'].hist()

In [ ]:
# histogram of DTI for non-foreclosed loans
all_loans[all_loans.foreclosure_flag == 0].loc[:, 'orig_dti'].hist()

In [ ]:
# histogram of DTI for foreclosed loans
all_loans[all_loans.foreclosure_flag == 1].loc[:, 'orig_dti'].hist()

Let's take a look at how foreclosure factors vary across companies.

In [ ]:
# frequency count of sellers with most foreclosed loans
all_loans[all_loans.foreclosure_flag == 1]['seller_name'].value_counts().plot.bar()

In [ ]:
# foreclosed loans aggregates
all_loans_aggs = pd.value_counts(all_loans['seller_name']).reset_index()
all_loans_aggs.columns = ['seller_name', 'total_loans']


# foreclosed loans aggregates
foreclosed_aggs = pd.value_counts(all_loans[all_loans.foreclosure_flag == 1]['seller_name']).reset_index()
foreclosed_aggs.columns = ['seller_name', 'total_foreclosed']

# compile results
all_loans_aggs = foreclosed_aggs.merge(all_loans_aggs)
all_loans_aggs['pct_foreclosed'] = all_loans_aggs['total_foreclosed']/all_loans_aggs['total_loans']
all_loans_aggs.sort_values(by=['pct_foreclosed'], ascending=False)

In [ ]:
# add columns for mean credit scores
all_loans_aggs = all_loans_aggs.merge(all_loans.groupby(['seller_name'])[['credit_score']].mean().reset_index())
all_loans_aggs = all_loans_aggs.rename(columns={'credit_score': 'mean_credit_all'})
all_loans_aggs = all_loans_aggs.merge(all_loans[all_loans.foreclosure_flag == 1].groupby(['seller_name'])[['credit_score']].mean().reset_index())
all_loans_aggs = all_loans_aggs.rename(columns={'credit_score': 'mean_credit_foreclosed'})


# add columns for median credit scores
all_loans_aggs = all_loans_aggs.merge(all_loans.groupby(['seller_name'])[['credit_score']].median().reset_index())
all_loans_aggs = all_loans_aggs.rename(columns={'credit_score': 'median_credit_all'})
all_loans_aggs = all_loans_aggs.merge(all_loans[all_loans.foreclosure_flag == 1].groupby(['seller_name'])[['credit_score']].median().reset_index())
all_loans_aggs = all_loans_aggs.rename(columns={'credit_score': 'median_credit_foreclosed'})


all_loans_aggs.sort_values(by='median_credit_foreclosed')